# Minimum Norm Interpolant

In [1]:
import numpy as np
import jax.numpy as jnp
import lineax as lx
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS


$n>p$ dgp, OLS solution not unique

In [2]:
def sparse_dgp(n = 10_000, p = 20_000, eta = 0.1):
    X = np.c_[np.repeat(1, n),
            np.random.normal(size = n*p).reshape((n, p))
        ]
    # initialize coef vector
    β, nzcount = np.repeat(0.0, p + 1), int(eta * p)
    # choose nzcount number of non-zero coef
    nzid = np.random.choice(p, nzcount, replace=False)
    # set them to random values
    β[nzid] = np.random.randn(nzcount)
    # build in heteroskedasticity
    e = np.random.normal(0, 0.5 + (0.1 * X[:, 1]>0), n)
    # generate y
    y = X @ β + e
    return y, X

y, X = sparse_dgp()


### statsmodels

In [3]:
%%time
smols = OLS(y, X).fit()

CPU times: user 1h 24min 14s, sys: 34min 40s, total: 1h 58min 55s
Wall time: 2min 6s


In [4]:
np.linalg.norm(smols.params)

32.06474491647644

Statsmodels is very slow with such problems.

### scikit

In [7]:
%%time
m = LinearRegression()
m.fit(X, y)
(y - m.predict(X)).max()


CPU times: user 35min 26s, sys: 13min 45s, total: 49min 11s
Wall time: 51.5 s


1.794120407794253e-12

In [8]:
np.linalg.norm(m.coef_)


32.063915612235505

### lineax

Very fast least squares solver (including for minimum norm interpolation problems). 


In [5]:
%%time
sol = lx.linear_solve(                                    # solve # Ax = b
        operator = lx.MatrixLinearOperator(jnp.array(X)), # A
        vector = jnp.array(y),                            # b
        solver=lx.AutoLinearSolver(well_posed=None),      
    )

betahat = sol.value
# does it interpolate
(y - X @ betahat).max()

CPU times: user 10min 31s, sys: 3min 35s, total: 14min 6s
Wall time: 18.9 s


Array(0.00014114, dtype=float32)

In [6]:
np.linalg.norm(betahat)


32.064747